# Homework 3 - Application Building

- Python Computing for Data Science (2022)

- Due Tuesday Feb 15 (8pm)

## CalCalc

Write a module called `CalCalc`, with a method called `calculate` that evaluates any string passed to it, and can be used from either the command line (using `argparse` with reasonable flags) or imported within Python. Feel free to use something like `eval()`, but be aware of some of the nasty things it can do, and make sure it doesn’t have too much power:  http://nedbatchelder.com/blog/201206/eval_really_is_dangerous.html. Perhaps explore the use of `numexpr` to constrain the landscape of possible uses to math expressions.

EXAMPLE:
```bash
$ python CalCalc.py -s '34*28'
$ 952
```
 AND, from within Python
 
```python
>>> from CalCalc import calculate
>>> calculate('34*20')
>>> 952
```

In [16]:
%%writefile CalCalc.py
# %%writefile calcalc/CalCalc.py
import numexpr as ne
import urllib.request

appID = 'JWKAQ4-U2LKHL8YEE'


def eval_wolfram(s, return_float=True):
    s = s.replace(' ','+')
    f = urllib.request.urlopen(f'http://api.wolframalpha.com/v1/result?appid={appID}&i={s}%3F&units=metric')
    result = str(f.peek(),"utf-8")
    result = result.replace('about ','') # remove the `about`

    # convert result to a float
    if return_float: 
        result = result.replace(' times 10 to the ','e') # put in scientific notation
        # replace common words with numerical value
        sub_words = {' thousand':'e3', ' million':'e6', ' billion':'e8', ' trillion':'e9'}
        for key in sub_words.keys():
            result = result.replace(key, sub_words[key])
        return(float(result.split(' ')[0]))

    # or return the string
    else: return(result)

def calculate(s, run_python=False, run_wolfram=False, return_float=True):
    '''
    use numexpr to evaluate string expression
    return result in desired format
    '''
    if run_python:
        try: 
            return(ne.evaluate(s).item())
        except Exception as e:
            print("Oops!", e.__class__, "occurred.")
            return("Perhaps try running with Wolfram (-w flag)")
    
    elif run_wolfram:
        return eval_wolfram(s, return_float)
    
    # if for some reason both python and wolfram flags are set, try evaluating
    # in python. If test fails, evaluate in wolfram
    else: 
        try: 
            return(ne.evaluate(s).item())
        except:
            return eval_wolfram(s, return_float)
        
if __name__ == '__main__':
    import argparse
    
    # parse command line arguments
    parser = argparse.ArgumentParser(description='Evaluate a string.')
    
    # define input argument
    parser.add_argument('string',
                       help='String to be evaulated')
    
    # -s will try to evaluate using python
    parser.add_argument('-s', action='store_true', default=False,
                        dest='run_python',
                        help='Run in python?')
    
    # -w will try to evaluate using wolfram
    parser.add_argument('-w', action='store_true', default=False,
                        dest='run_wolfram',
                        help='Send to wolfram?')
    # add versioning
    parser.add_argument('--version', action='version', version='%(prog)s 1.0')

    
    results = parser.parse_args()
    print(calculate(results.string, run_python=results.run_python, run_wolfram=results.run_wolfram))
    

Overwriting CalCalc.py


In [5]:
# show it works from command line
!python CalCalc.py -s '34*28'

952


In [4]:
# show it works form within python
from CalCalc import calculate
calculate('34*28')

952

### Add Wolfram

To make this more awesome, have your function interact with the Wolfram|Alpha API to ask it what it thinks of the difficult questions.  To make this work, experiment with `urllib` and a URL like this:
'http://api.wolframalpha.com/v2/query?input=XXXXX&appid=YYYYY'
where you replace the XXXXX with what you want to know.  NOTE: the ‘&appid=YYYYY’ part is vital; it is my W|A AppID.  You should get your own app ID from Wolfram by following the instructions here: https://products.wolframalpha.com/simple-api/documentation/. You will need a wolfram account to be able to access their products, but it doesn't need to be a paid account for the app id.
EXAMPLE:

```bash
$ python CalCalc.py -w 'mass of the moon in kg'
7.3459e+22
```

AND, from within Python

```python
>>> from CalCalc import calculate
>>> calculate('mass of the moon in kg',  return_float=True) * 10
>>> 7.3459e+23
```

We are requesting that your `CalCalc.py` script should be able to evaluate some inputs locally in python (Example 1 from the homework), and some remotely in wolfram (Example 2 from the homework) all from within the same script file, even if the first example could technically be achieved within Wolfram as well.

In [19]:
# form command line
!python CalCalc.py -w 'mass of the moon in kg'

7.3459e+22


In [20]:
# from within Python
# from CalCalc import calculate

calculate('mass of the moon in kg',  return_float=True) * 10

7.3459e+23

## Adding it to Github

Start a github project for CalCalc. Include a setup.py, README.txt, LICENSE.txt, MANIFEST.in, etc. and turn your module into a proper Python Distribution, so that we can install it and use it. See https://packaging.python.org/en/latest/guides/distributing-packages-using-setuptools/ 

Example Folder Hierarchy:
```bash
Your_Homework3_Folder/calcalc
                      |--> CalCalc.py
                      |--> __init__.py
Your_Homework3_Folder/setup.py
Your_Homework3_Folder/README.txt
...
```
Include at least 5 test functions in CalCalc.py, and test with `pytest`, to make sure it behaves the way you think it should.

EXAMPLE `CalCalc.py`:
```python
# ...
def calculate([...]):
    [...]

def test_1():
    assert abs(4. - calculate('2**2')) < 0.001
```

When grading, we will create a virtual environment and attempt to install your module by running:

```bash
pip install build
```

In [18]:
%%writefile setup.py
from setuptools import setup, find_packages
import pathlib

here = pathlib.Path(__file__).parent.resolve()

# Get the long description from the README file
long_description = (here / 'README.md').read_text(encoding='utf-8')

# Arguments marked as "Required" below must be included for upload to PyPI.
# Fields marked as "Optional" may be commented out.

setup(
    # This is the name of your project. The first time you publish this
    # package, this name will be registered for you. It will determine how
    # users can install this project, e.g.:
    #
    # $ pip install sampleproject
    #
    # And where it will live on PyPI: https://pypi.org/project/sampleproject/
    #
    # There are some restrictions on what makes a valid project name
    # specification here:
    # https://packaging.python.org/specifications/core-metadata/#name
    name='calcalc',  # Required

    # Versions should comply with PEP 440:
    # https://www.python.org/dev/peps/pep-0440/
    #
    # For a discussion on single-sourcing the version across setup.py and the
    # project code, see
    # https://packaging.python.org/guides/single-sourcing-package-version/
    version='1.0',  # Required

    # This is a one-line description or tagline of what your project does. This
    # corresponds to the "Summary" metadata field:
    # https://packaging.python.org/specifications/core-metadata/#summary
    description='Evaluate string using python and/or wolfram',  # Optional

    # This is an optional longer description of your project that represents
    # the body of text which users will see when they visit PyPI.
    #
    # Often, this is the same as your README, so you can just read it in from
    # that file directly (as we have already done above)
    #
    # This field corresponds to the "Description" metadata field:
    # https://packaging.python.org/specifications/core-metadata/#description-optional
    long_description=long_description,  # Optional

    # Denotes that our long_description is in Markdown; valid values are
    # text/plain, text/x-rst, and text/markdown
    #
    # Optional if long_description is written in reStructuredText (rst) but
    # required for plain-text or Markdown; if unspecified, "applications should
    # attempt to render [the long_description] as text/x-rst; charset=UTF-8 and
    # fall back to text/plain if it is not valid rst" (see link below)
    #
    # This field corresponds to the "Description-Content-Type" metadata field:
    # https://packaging.python.org/specifications/core-metadata/#description-content-type-optional
    long_description_content_type='text/markdown',  # Optional (see note above)

    # This should be a valid link to your project's main homepage.
    #
    # This field corresponds to the "Home-Page" metadata field:
    # https://packaging.python.org/specifications/core-metadata/#home-page-optional
    url='https://github.com/alizabeverage/python-ay250-homework/hw3/calcalc',  # Optional

    # This should be your name or the name of the organization which owns the
    # project.
    author='Aliza Beverage',  # Optional

    # This should be a valid email address corresponding to the author listed
    # above.
    author_email='abeverage@berkeley.edu',  # Optional

    # Classifiers help users find your project by categorizing it.
    #
    # For a list of valid classifiers, see https://pypi.org/classifiers/
    # classifiers=[  # Optional
    #     # How mature is this project? Common values are
    #     #   3 - Alpha
    #     #   4 - Beta
    #     #   5 - Production/Stable
    #     'Development Status :: 3 - Alpha',

    #     # Indicate who your project is intended for
    #     'Intended Audience :: Developers',
    #     'Topic :: Software Development :: Build Tools',

    #     # Pick your license as you wish
    #     'License :: OSI Approved :: MIT License',

    #     # Specify the Python versions you support here. In particular, ensure
    #     # that you indicate you support Python 3. These classifiers are *not*
    #     # checked by 'pip install'. See instead 'python_requires' below.
    #     'Programming Language :: Python :: 3',
    #     'Programming Language :: Python :: 3.6',
    #     'Programming Language :: Python :: 3.7',
    #     'Programming Language :: Python :: 3.8',
    #     'Programming Language :: Python :: 3.9',
    #     "Programming Language :: Python :: 3.10",
    #     'Programming Language :: Python :: 3 :: Only',
    # ],

    # This field adds keywords for your project which will appear on the
    # project page. What does your project relate to?
    #
    # Note that this is a list of additional keywords, separated
    # by commas, to be used to assist searching for the distribution in a
    # larger catalog.
    # keywords='sample, setuptools, development',  # Optional

    # When your source code is in a subdirectory under the project root, e.g.
    # `src/`, it is necessary to specify the `package_dir` argument.
    # package_dir={'': 'src'},  # Optional

    # You can just specify package directories manually here if your project is
    # simple. Or you can use find_packages().
    #
    # Alternatively, if you just want to distribute a single Python file, use
    # the `py_modules` argument instead as follows, which will expect a file
    # called `my_module.py` to exist:
    #
    py_modules=["calcalc"],
    #
    # packages=find_packages(where='src'),  # Required

    # Specify which Python versions you support. In contrast to the
    # 'Programming Language' classifiers above, 'pip install' will check this
    # and refuse to install the project if the version does not match. See
    # https://packaging.python.org/guides/distributing-packages-using-setuptools/#python-requires
    python_requires='>=3.6, <4',

    # This field lists other packages that your project depends on to run.
    # Any package you put here will be installed by pip when your project is
    # installed, so they must be valid existing projects.
    #
    # For an analysis of "install_requires" vs pip's requirements files see:
    # https://packaging.python.org/discussions/install-requires-vs-requirements/
    install_requires=['numexpr'],  # Optional

    # List additional groups of dependencies here (e.g. development
    # dependencies). Users will be able to install these using the "extras"
    # syntax, for example:
    #
    #   $ pip install sampleproject[dev]
    #
    # Similar to `install_requires` above, these must be valid existing
    # projects.
    # extras_require={  # Optional
    #     'dev': ['check-manifest'],
    #     'test': ['coverage'],
    # },

    # If there are data files included in your packages that need to be
    # installed, specify them here.
    # package_data={  # Optional
    #     'sample': ['package_data.dat'],
    # },

    # Although 'package_data' is the preferred approach, in some case you may
    # need to place data files outside of your packages. See:
    # http://docs.python.org/distutils/setupscript.html#installing-additional-files
    #
    # In this case, 'data_file' will be installed into '<sys.prefix>/my_data'
    # data_files=[('my_data', ['data/data_file'])],  # Optional

    # To provide executable scripts, use entry points in preference to the
    # "scripts" keyword. Entry points provide cross-platform support and allow
    # `pip` to create the appropriate form of executable for the target
    # platform.
    #
    # For example, the following would provide a command called `sample` which
    # executes the function `main` from this package when invoked:
    entry_points={  # Optional
        'console_scripts': [
            'search=search:main',
        ],
    },

    # List additional URLs that are relevant to your project as a dict.
    #
    # This field corresponds to the "Project-URL" metadata fields:
    # https://packaging.python.org/specifications/core-metadata/#project-url-multiple-use
    #
    # Examples listed include a pattern for specifying where the package tracks
    # issues, where the source is hosted, where to say thanks to the package
    # maintainers, and where to support the project financially. The key is
    # what's used to render the link text on PyPI.
    # project_urls={  # Optional
    #     'Bug Reports': 'https://github.com/pypa/sampleproject/issues',
    #     'Funding': 'https://donate.pypi.org',
    #     'Say Thanks!': 'http://saythanks.io/to/example',
    #     'Source': 'https://github.com/pypa/sampleproject/',
    # },
)

Overwriting setup.py


In [19]:
!python -m pip install -e .

Obtaining file:///Users/alizabeverage/Class/python-ay250-homework/hw3
  Running setup.py develop for calcalc


PKG-INFO              dependency_links.txt  requires.txt
SOURCES.txt           entry_points.txt      top_level.txt


### CalCalc on CI

Get your project working with GitHub Actions and make sure your tests are run and pass. Give us a link to you GH actions for your site here (e.g. https://github.com/profjsb/PyAdder/actions):

In [16]:
mv README.txt README.md

### **(Bonus/Extra Credit)** 

  Get your project working on Azure, AWS or Google Compute Cloud with a Flask front-end. You can use the example from class as a template. Start a VM on one of these PaaS. A user should be able to submit their calcalc query on a form (hosted on your VM) and get the result back.

You should be able to add an `app.py` (with Flask) into your CalCalc project. Be sure to open up the port on the VM that you are serving on. Let us know the URL to your app here: